In [10]:
from utils import *
from massfunction import *

import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import os
import emcee
import sys
import numpy as np
import pickle

In [11]:
cosmos_f = open('data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

In [12]:
X = []
Y = []
idx = {}
cidx = 0
weird_boxes = ['Box63_1400', 'Box35_1400', 'Box_n50_38_1400']
for box in cosmo_params:
    if(box in weird_boxes):
        continue
    X += [box]
    idx[box] = cidx
    cidx+=1
    Y += [[cosmo_params[box][key] for key in cosmo_params[box]]]
    
X = np.array(X)
Y = np.array(Y)

from sklearn.preprocessing import StandardScaler

# Assuming Y is your input matrix with shape (n_samples, n_features)
# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(Y)

# Standardize the data
Y = scaler.transform(Y)

In [13]:
print(len(X))

147


In [14]:
def find_most_central_datapoint_index(Y):
    centroid = np.mean(Y, axis=0)
    distances = np.linalg.norm(Y - centroid, axis=1)
    most_central_index = np.argmin(distances)
    return most_central_index
X[find_most_central_datapoint_index(Y)]

'Box_n50_0_1400'

In [22]:
def dist(a,b):
    return np.sqrt(np.sum(np.square(np.array(Y[b])-np.array(Y[a]))))

edges = [[dist(i,j) for i in range(len(X))] for j in range(len(X))]

In [23]:
import sys

def find_min_key(key, mst_set, n):
    min_key = sys.maxsize
    min_index = -1
    for i in range(n):
        if key[i] < min_key and not mst_set[i]:
            min_key = key[i]
            min_index = i
    return min_index

def prim_mst(edges, n):
    parent = [None] * n
    key = [sys.maxsize] * n
    mst_set = [False] * n

    key[0] = 0  # Start with the first node as the root
    parent[0] = -1  # Root has no parent

    for _ in range(n - 1):
        u = find_min_key(key, mst_set, n)
        mst_set[u] = True

        for v in range(n):
            if edges[u][v] > 0 and not mst_set[v] and edges[u][v] < key[v]:
                key[v] = edges[u][v]
                parent[v] = u

    mst = [[0] * n for _ in range(n)]
    for i in range(1, n):
        mst[i][parent[i]] = edges[i][parent[i]]
        mst[parent[i]][i] = edges[parent[i]][i]

    return mst

n = len(edges)

mst = prim_mst(edges, n)

In [24]:
from collections import deque

def bfs_traversal(mst, start):
    tot = 0
    n = len(mst)
    visited = [False] * n
    parent = [None] * n

    queue = deque()
    queue.append(start)
    visited[start] = True

    while queue:
        node = queue.popleft()
        print("%-10s"%(X[node]), end='\t ')
        if parent[node] is not None:
            prev = parent[node]
            dist = np.sqrt(np.sum(np.square(np.array(Y[idx[X[node]]])- np.array(Y[idx[X[prev]]]))))
            tot += dist

            print("Prev: %-10s\t distance:%.2f"%(X[parent[node]], dist),  end='')
        print()

        for neighbor in range(n):
            if mst[node][neighbor] > 0 and not visited[neighbor]:
                queue.append(neighbor)
                visited[neighbor] = True
                parent[neighbor] = node
    assert(all(visited))
    print(tot)

start_node = idx['Box_n50_0_1400'] # Starting node for traversal

bfs_traversal(mst, start_node)


Box_n50_0_1400	 
Box_n50_4_1400	 Prev: Box_n50_0_1400	 distance:1.23
Box_n50_14_1400	 Prev: Box_n50_0_1400	 distance:1.06
Box_n50_21_1400	 Prev: Box_n50_0_1400	 distance:1.18
Box_n50_33_1400	 Prev: Box_n50_0_1400	 distance:1.07
Box_n50_35_1400	 Prev: Box_n50_0_1400	 distance:1.07
Box_n50_43_1400	 Prev: Box_n50_0_1400	 distance:1.27
Box_n50_17_1400	 Prev: Box_n50_4_1400	 distance:1.28
Box_n50_34_1400	 Prev: Box_n50_4_1400	 distance:1.12
Box3_1400 	 Prev: Box_n50_14_1400	 distance:1.52
Box_n50_20_1400	 Prev: Box_n50_14_1400	 distance:1.70
Box_n50_36_1400	 Prev: Box_n50_21_1400	 distance:0.87
Box_n50_19_1400	 Prev: Box_n50_33_1400	 distance:1.21
Box_n50_37_1400	 Prev: Box_n50_33_1400	 distance:0.79
Box_n50_46_1400	 Prev: Box_n50_33_1400	 distance:1.31
Box_n50_1_1400	 Prev: Box_n50_35_1400	 distance:0.97
Box_n50_26_1400	 Prev: Box_n50_35_1400	 distance:1.16
Box_n50_6_1400	 Prev: Box_n50_43_1400	 distance:1.09
Box_n50_32_1400	 Prev: Box_n50_43_1400	 distance:1.09
Box_n50_44_1400	 Prev: Box_

In [26]:
dist(idx['Box22_1400'], idx['Box54_1400'])

3.4292828115946183

In [33]:
sorted(edges[idx['Box22_1400']])

[0.0,
 3.4292828115946183,
 3.5368170133922345,
 3.679179082852491,
 3.703445953399301,
 3.75291211023859,
 3.7666437979117435,
 3.9438077056600127,
 3.989081506718773,
 4.033624273625493,
 4.0446369262250945,
 4.092679193646477,
 4.108825462302852,
 4.123794114954355,
 4.185764731039315,
 4.240368260045832,
 4.252921568744867,
 4.2640307993235504,
 4.274931103345747,
 4.2861909844902835,
 4.347524820680633,
 4.39987878905104,
 4.422593669043359,
 4.423780674045245,
 4.4258493367425125,
 4.458152464803018,
 4.4728734819215985,
 4.505697175407108,
 4.5087162799769676,
 4.511396029894333,
 4.514558006839311,
 4.541569793615992,
 4.565715413632812,
 4.566404787570421,
 4.574970814159601,
 4.58311997808539,
 4.585501601673708,
 4.590066418244965,
 4.617250458779258,
 4.6188333250969515,
 4.6526223691063535,
 4.668341103626374,
 4.689155623664019,
 4.709952117817649,
 4.730764133347209,
 4.731219166680071,
 4.749693979096804,
 4.753670995505437,
 4.754109302199875,
 4.757364340521435,
 4.75

In [40]:
from collections import deque

import subprocess
import time

def run_fit_iter(box, prev_box):
    #run the fit job for this guy 
    subprocess.run('./scripts/fit_individ_handler.sh %s %s'%(box, prev_box), shell=True)

def bfs_traversal_run_jobs(mst, start):
    n = len(mst)
    visited = [False] * n
    parent = [None] * n

    queue = deque()
    queue.append(start)
    visited[start] = True

    while queue:
#         print('----START-----')
        time.sleep(20)
        node = queue.popleft()
        
        
#         print("%-10s"%(X[node]), end='\t ')
#         if parent[node] is not None:
#             prev = parent[node]
#             dist = np.sqrt(np.sum(np.square(np.array(Y[idx[X[node]]])- np.array(Y[idx[X[prev]]]))))

#             print("Prev: %-10s\t distance:%.2f"%(X[parent[node]], dist),  end='')
#         print()

        #check if parent still running
        curr_parent = parent[node]
        if(curr_parent is not None):
            squeue_output = subprocess.check_output(['squeue', '-u', 'delon', '-n', 'fit-iter-handler-%s'%(X[curr_parent]), '-h']).decode().strip()
#             print('sq', squeue_output)
            if not squeue_output: #if job done running 
                print('Running fit for %s'%(X[node]))
                run_fit_iter(X[node], X[curr_parent])
                #add its neighboros to the queue
                for neighbor in range(n):
                    if mst[node][neighbor] > 0 and not visited[neighbor]:
                        queue.append(neighbor)
                        visited[neighbor] = True
                        parent[neighbor] = node
            else: #parent job not done running
#                 print('Fit for %s still running'%(X[curr_parent]))
                queue.append(node)
        else:
            for neighbor in range(n):
                if mst[node][neighbor] > 0 and not visited[neighbor]:
                    queue.append(neighbor)
                    visited[neighbor] = True
                    parent[neighbor] = node
#         print('-----END----')

    assert(all(visited))

start_node = idx['Box_n50_0_1400'] # Starting node for traversal

bfs_traversal_run_jobs(mst, start_node)


----START-----
Box_n50_0_1400	 
-----END----
----START-----
Box_n50_4_1400	 Prev: Box_n50_0_1400	 distance:1.23
sq 
Running fit for Box_n50_4_1400
scripts/fit-iter-handler-Box_n50_4_1400.sh
Submitted batch job 21981583
-----END----
----START-----
Box_n50_14_1400	 Prev: Box_n50_0_1400	 distance:1.06
sq 
Running fit for Box_n50_14_1400
scripts/fit-iter-handler-Box_n50_14_1400.sh
Submitted batch job 21981596
-----END----
----START-----
Box_n50_21_1400	 Prev: Box_n50_0_1400	 distance:1.18
sq 
Running fit for Box_n50_21_1400
scripts/fit-iter-handler-Box_n50_21_1400.sh
Submitted batch job 21981601
-----END----
----START-----
Box_n50_33_1400	 Prev: Box_n50_0_1400	 distance:1.07
sq 
Running fit for Box_n50_33_1400
scripts/fit-iter-handler-Box_n50_33_1400.sh
Submitted batch job 21981620
-----END----
----START-----
Box_n50_35_1400	 Prev: Box_n50_0_1400	 distance:1.07
sq 
Running fit for Box_n50_35_1400
scripts/fit-iter-handler-Box_n50_35_1400.sh
Submitted batch job 21981643
-----END----
----STAR

Box60_1400	 Prev: Box5_1400 	 distance:1.96
sq 
Running fit for Box60_1400
scripts/fit-iter-handler-Box60_1400.sh
Submitted batch job 21982006
-----END----
----START-----
Box52_1400	 Prev: Box4_1400 	 distance:2.29
sq 
Running fit for Box52_1400
scripts/fit-iter-handler-Box52_1400.sh
Submitted batch job 21982022
-----END----
----START-----
Box30_1400	 Prev: Box_n50_15_1400	 distance:1.93
sq 
Running fit for Box30_1400
scripts/fit-iter-handler-Box30_1400.sh
Submitted batch job 21982033
-----END----
----START-----
Box_n50_10_1400	 Prev: Box_n50_15_1400	 distance:1.42
sq 
Running fit for Box_n50_10_1400
scripts/fit-iter-handler-Box_n50_10_1400.sh
Submitted batch job 21982060
-----END----
----START-----
Box_n50_30_1400	 Prev: Box_n50_15_1400	 distance:1.59
sq 
Running fit for Box_n50_30_1400
scripts/fit-iter-handler-Box_n50_30_1400.sh
Submitted batch job 21982061
-----END----
----START-----
Box77_1400	 Prev: Box91_1400	 distance:1.69
sq 
Running fit for Box77_1400
scripts/fit-iter-handler-

sq 
Running fit for Box55_1400
scripts/fit-iter-handler-Box55_1400.sh
Submitted batch job 21982501
-----END----
----START-----
Box27_1400	 Prev: Box88_1400	 distance:1.85
sq 
Running fit for Box27_1400
scripts/fit-iter-handler-Box27_1400.sh
Submitted batch job 21982517
-----END----
----START-----
Box26_1400	 Prev: Box1_1400 	 distance:0.99
sq 
Running fit for Box26_1400
scripts/fit-iter-handler-Box26_1400.sh
Submitted batch job 21982537
-----END----
----START-----
Box44_1400	 Prev: Box1_1400 	 distance:2.13
sq 
Running fit for Box44_1400
scripts/fit-iter-handler-Box44_1400.sh
Submitted batch job 21982547
-----END----
----START-----
Box34_1400	 Prev: Box_n50_27_1400	 distance:1.41
sq 
Running fit for Box34_1400
scripts/fit-iter-handler-Box34_1400.sh
Submitted batch job 21982571
-----END----
----START-----
Box_n50_40_1400	 Prev: Box_n50_27_1400	 distance:1.50
sq 
Running fit for Box_n50_40_1400
scripts/fit-iter-handler-Box_n50_40_1400.sh
Submitted batch job 21982574
-----END----
----STAR

Submitted batch job 21983343
-----END----
----START-----
Box70_1400	 Prev: Box41_1400	 distance:1.44
sq 
Running fit for Box70_1400
scripts/fit-iter-handler-Box70_1400.sh
Submitted batch job 21983364
-----END----
----START-----
Box13_1400	 Prev: Box95_1400	 distance:1.51
sq 21983272     kipac fit-iter    delon  R       0:11      1 sh02-03n72
Fit for Box95_1400 still running
-----END----
----START-----
Box23_1400	 Prev: Box_n50_23_1400	 distance:1.75
sq 21983298     kipac fit-iter    delon  R       0:16      1 sh02-03n72
Fit for Box_n50_23_1400 still running
-----END----
----START-----
Box85_1400	 Prev: Box70_1400	 distance:2.37
sq 21983364     kipac fit-iter    delon PD       0:00      1 (Priority)
Fit for Box70_1400 still running
-----END----
----START-----
Box13_1400	 Prev: Box95_1400	 distance:1.51
sq 
Running fit for Box13_1400
scripts/fit-iter-handler-Box13_1400.sh
Submitted batch job 21983395
-----END----
----START-----
Box23_1400	 Prev: Box_n50_23_1400	 distance:1.75
sq 
Running